# Import libraries and load datasets

In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

In [2]:
# Import disaster_messages as a df
messages = pd.read_csv('data\\01_raw\\disaster_messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [3]:
# Import disaster_categories as a df
categories = pd.read_csv('data\\01_raw\\disaster_categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [4]:
#Merge the two dataframes on id
df = messages.merge(categories, on='id')
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [5]:
# Split the categories column into separate columns
split_categories = df['categories'].str.split(';', expand=True)

# Create a list of column names for categories
category_colnames = [x.split('-')[0] for x in split_categories.iloc[0, :]]

# Rename the columns of `categories`
split_categories.columns = category_colnames

# Concatenate the 'id' column with the split categories
result = pd.concat([df['id'], split_categories], axis=1)

result.head()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
1,7,related-1,request-0,offer-0,aid_related-1,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-1,floods-0,storm-1,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
2,8,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
3,9,related-1,request-1,offer-0,aid_related-1,medical_help-0,medical_products-1,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0
4,12,related-1,request-0,offer-0,aid_related-0,medical_help-0,medical_products-0,search_and_rescue-0,security-0,military-0,...,aid_centers-0,other_infrastructure-0,weather_related-0,floods-0,storm-0,fire-0,earthquake-0,cold-0,other_weather-0,direct_report-0


In [6]:
def convert_categories(df):
    for column in df:
        if column == 'id':
            continue
        # set each value to be the last character of the string
        df.loc[:, column] = df[column].str[-1]
        # convert column from string to numeric
        df.loc[:, column] = pd.to_numeric(df[column])
    return df

# Apply the function to the result dataframe
result = convert_categories(result)
result.head()

,id,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,0,0,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,1,1,0,1,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,12,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
# Drop the original categories column from `df`
df = df.drop('categories', axis=1)

#Join the result dataframe with the original dataframe
df = df.merge(result, on='id')
df.head(n=2)

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [8]:
# Print the shape of the DataFrame before removing duplicates
print(f'Original DataFrame shape: {df.shape}')

# Check the number of duplicates
print(f'Number of duplicates: {df.duplicated().sum()}')

# Drop duplicates
df = df.drop_duplicates()

# Print the shape of the DataFrame after removing duplicates
print(f'DataFrame shape after removing duplicates: {df.shape}')

# Check to see if there are any remaining duplicates
print(f'Number of duplicates after removal: {df.duplicated().sum()}')

# Print the percent of columns dropped and round to two decimal places
print(f'Percent of columns dropped: {round(1 - df.shape[0] / result.shape[0], 2)}%')

df.head(n=2)

Original DataFrame shape: (27250, 40)
Number of duplicates: 1034
DataFrame shape after removing duplicates: (26216, 40)
Number of duplicates after removal: 0
Percent of columns dropped: 0.01%


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0


In [9]:
# Check the df for any missing values
print(f'Number of missing values: {df.isnull().sum().sum()}')

# Show me the columns with missing values
print(f'Columns with missing values: {df.columns[df.isnull().any()]}')

#Excluding the original message column, check for any missing values
print(f'Number of missing values in the original message column: {df.message.isnull().sum()}')

Number of missing values: 16046
Columns with missing values: Index(['original'], dtype='object')
Number of missing values in the original message column: 0


In [10]:
# Save the clean dataset to a .csv
df.to_csv('data\\02_stg\\stg_disaster_messages.csv', index=False)

# Save the clean dataset to a sqlite database.
# To Do: write this code so that the table name is the same as the file name minus the extension
engine = create_engine('sqlite:///data/02_stg//stg_disaster_response.db')
df.to_sql('stg_disaster_response', engine, index=False, if_exists='replace')

26216